In [1]:
import pandas as pd
import json
from collections import defaultdict
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
df = pd.read_csv('../data/api-info.csv')
df.db_name = df.db_name.fillna('')
df.report_name = df.report_name.fillna('')
df.main_view = df.main_view.fillna('')
df.table_1 = df.table_1.fillna('')
df.table_2 = df.table_2.fillna('')
df.table_3 = df.table_3.fillna('')
# df.tab_impact = df.tab_impact.fillna('')
df.lookback = df.lookback.fillna(0)
# df.lookback = df.lookback.replace('year-to-date', 100)
df

,source,source_name,db_name,report_name,lookback,main_view,table_1,table_2,table_3
0,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Standard,14,[dbo].[vw_StandardReport],,,
1,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Standard,14,[dbo].[FactStandardReport],,,
2,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Activity,30,[dbo].[vw_Floodlight],,,
3,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Activity,30,[dbo].[FactFloodlight],,,
4,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Monthly Reach,365,[dbo].[vw_CookieRNF_Month],,,
...,...,...,...,...,...,...,...,...,...
317,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[foursquare].[dim_advertiser],,,
318,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[foursquare].[fact_reports],[foursquare].[dim_reports],,
319,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[dbo].[vw_FourSquare_Overall_Master],,,
320,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[foursquare].[dim_advertiser],,,


In [5]:
def find_idx_or_add_new_child(L, key, value):
    for i, line in enumerate(L):
        if line[key] == value:
            return i
    L.append({key: value, 'name': value, 'children': []})
    return len(L)-1


nested = []
for index, row in df.iterrows():
    expanding = nested
    for col in df.columns:
        if 'table' in col and row[col] == '':
            break
        idx = find_idx_or_add_new_child(expanding, col, row[col])
        expanding = expanding[idx]['children']

In [7]:
json.dump(nested, open('../data/nested.json', 'w'))